In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, Layer, RNN
from tensorflow.keras.models import Model, Sequential
import pennylane as qml

TF_ENABLE_ONEDNN_OPTS=2

# ======= 1. Prétraitement et préparation des données =======
print("\n=== 1. Chargement et nettoyage ===")
df = pd.read_csv('data/Hamilton.csv', skiprows=1)
df = df[pd.to_datetime(df['date'], errors='coerce').notnull()].reset_index(drop=True)
df = df.drop(columns=['time'])
df = df.rename(columns={'wlvalue': 'water_level', 'flvalue': 'flow'})
df['date'] = pd.to_datetime(df['date'])

# Normalisation et PCA
scaler_hydro = MinMaxScaler()
hydro_norm = scaler_hydro.fit_transform(df[['water_level', 'flow']])
pca = PCA(n_components=1)
df['hydro_pca'] = pca.fit_transform(hydro_norm)
df['dayofyear'] = df['date'].dt.dayofyear
df['sin_doy'] = np.sin(2*np.pi*df['dayofyear']/365)
df['cos_doy'] = np.cos(2*np.pi*df['dayofyear']/365)
features = ['hydro_pca', 'sin_doy', 'cos_doy']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

WINDOW = 7
X, y = [], []
for i in range(WINDOW, len(df)-1):
    X.append(df[features].iloc[i-WINDOW:i].values)
    y.append(df['water_level'].iloc[i+1])
X, y = np.array(X), np.array(y)

N = len(X)
Ntrain = int(0.7*N)
Nval = int(0.15*N)
X_tr, X_val, X_te = X[:Ntrain], X[Ntrain:Ntrain+Nval], X[Ntrain+Nval:]
y_tr, y_val, y_te = y[:Ntrain], y[Ntrain:Ntrain+Nval], y[Ntrain+Nval:]

# =============== 2. Scheduled Sampling ===============
def scheduled_sampling(X, y, model, scaler, features, prob=0.7):
    """Modifie la première feature (hydro_pca) des séquences X avec la prédiction du modèle, stochastiquement."""
    X_mod = X.copy()
    for i in range(X.shape[0]):
        for t in range(1, X.shape[1]):
            if np.random.rand() > prob:
                pred = model.predict(X_mod[i:i+1], verbose=0)[0][0]
                # Remise à l'échelle normalisée de la prédiction
                X_mod[i, t, 0] = scaler.transform(pd.DataFrame([[pred, 0, 0]], columns=features))[0][0]
    return X_mod

# =============== 3. Entraînement avec schedule ===============
def train_with_schedule(model, X, y, X_val, y_val, scaler, features, EPOCHS=10, BATCH_SIZE=16, schedule_decay=0.05, verbose=1):
    losses, maes, val_losses, val_maes = [], [], [], []
    sampling_prob = 1.0
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}, scheduled_sampling_prob={sampling_prob:.3f}")
        X_mod = scheduled_sampling(X, y, model, scaler, features, prob=sampling_prob)
        hist = model.fit(X_mod, y, epochs=1, batch_size=BATCH_SIZE, verbose=verbose, validation_data=(X_val, y_val))
        losses.append(hist.history['loss'][0])
        maes.append(hist.history['mae'][0])
        val_losses.append(hist.history['val_loss'][0])
        val_maes.append(hist.history['val_mae'][0])
        sampling_prob = max(0.0, sampling_prob - schedule_decay)
    return losses, maes, val_losses, val_maes

# ======= 4. LSTM Classique =======
print("\n=== LSTM Classique ===")
def build_lstm():
    model = Sequential([
        Input(shape=(WINDOW, len(features))),
        LSTM(32, return_sequences=False),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

lstm = build_lstm()
print(lstm.summary())
EPOCHS = 10
BATCH_SIZE = 16

print("\n--- Entraînement LSTM classique (avec scheduled sampling) ---")
losses_lstm, maes_lstm, vallosses_lstm, valmaes_lstm = train_with_schedule(
    lstm, X_tr, y_tr, X_val, y_val, scaler, features, EPOCHS=EPOCHS, BATCH_SIZE=BATCH_SIZE
)

# ======= 5. QLSTM (vrai LSTM quantique) =======
print("\n=== QLSTM (vrai LSTM quantique) ===")
N_QUBITS = 4      # 3 features + 1 hidden
N_LAYERS = 8

dev = qml.device("default.qubit", wires=N_QUBITS)

@qml.qnode(dev, interface='tf')
def qru_gate(inputs, weights):
    for l in range(N_LAYERS):
        for i in range(N_QUBITS):
            qml.RY(np.pi * inputs[i], wires=i)
            qml.RZ(weights[l, i], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(N_QUBITS)]

class QuantumLSTMCell(Layer):
    def __init__(self, units=1, n_qubits=N_QUBITS, n_layers=N_LAYERS, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.state_size = [units, units]
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.q_weights = [self.add_weight(
            shape=(n_layers, n_qubits), initializer="uniform", trainable=True, name=f'q_weights_{gate}'
        ) for gate in ['i','f','c','o']]
    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        concat = tf.concat([inputs, h_tm1], axis=-1)
        concat_circ = concat[:, :self.n_qubits]
        i = tf.map_fn(lambda x: tf.cast(tf.math.real(tf.convert_to_tensor(qru_gate(x, self.q_weights[0])[0], dtype=tf.float64)), tf.float32), concat_circ, fn_output_signature=tf.float32)
        f = tf.map_fn(lambda x: tf.cast(tf.math.real(tf.convert_to_tensor(qru_gate(x, self.q_weights[1])[0], dtype=tf.float64)), tf.float32), concat_circ, fn_output_signature=tf.float32)
        c_bar = tf.map_fn(lambda x: tf.cast(tf.math.real(tf.convert_to_tensor(qru_gate(x, self.q_weights[2])[0], dtype=tf.float64)), tf.float32), concat_circ, fn_output_signature=tf.float32)
        o = tf.map_fn(lambda x: tf.cast(tf.math.real(tf.convert_to_tensor(qru_gate(x, self.q_weights[3])[0], dtype=tf.float64)), tf.float32), concat_circ, fn_output_signature=tf.float32)
        i = tf.sigmoid(i)
        f = tf.sigmoid(f)
        o = tf.sigmoid(o)
        c_bar = tf.tanh(c_bar)
        c_t = f * c_tm1[:,0] + i * c_bar
        h_t = o * tf.tanh(c_t)
        h_t = tf.expand_dims(h_t, -1)
        c_t = tf.expand_dims(c_t, -1)
        return h_t, [h_t, c_t]

inputs = Input(shape=(WINDOW, len(features)))
cell = QuantumLSTMCell(units=1, n_qubits=4)
rnn = RNN(cell, return_sequences=False)
x = rnn(inputs)
output = Dense(1, activation='linear')(x)
lstm_qru = Model(inputs=inputs, outputs=output)
lstm_qru.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(lstm_qru.summary())

print("\n--- Entraînement QLSTM (avec scheduled sampling) ---")
losses_lstm_qru, maes_lstm_qru, vallosses_lstm_qru, valmaes_lstm_qru = train_with_schedule(
    lstm_qru, X_tr, y_tr, X_val, y_val, scaler, features, EPOCHS=EPOCHS, BATCH_SIZE=4
)

# ======= 6. Visualisation des courbes d'apprentissage =======
plt.figure(figsize=(12,6))
plt.plot(losses_lstm, label='Train Loss LSTM')
plt.plot(vallosses_lstm, label='Val Loss LSTM')
plt.plot(losses_lstm_qru, label='Train Loss QLSTM')
plt.plot(vallosses_lstm_qru, label='Val Loss QLSTM')
plt.title('Courbes de loss par epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

plt.figure(figsize=(12,6))
plt.plot(maes_lstm, label='Train MAE LSTM')
plt.plot(valmaes_lstm, label='Val MAE LSTM')
plt.plot(maes_lstm_qru, label='Train MAE QLSTM')
plt.plot(valmaes_lstm_qru, label='Val MAE QLSTM')
plt.title('Courbes de MAE par epoch')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

# ======= 7. Prédictions finales sur les 30 derniers jours =======
y_pred_lstm = lstm.predict(X_te[-30:])
y_pred_qru = lstm_qru.predict(X_te[-30:])
dates_last = df['date'].iloc[-30:]

print("\n--- Données vraies (30 derniers jours):", y_te[-30:])
print("--- LSTM predictions:", y_pred_lstm.flatten())
print("--- QLSTM predictions:", y_pred_qru.flatten())

plt.figure(figsize=(14,6))
plt.plot(dates_last, y_te[-30:], 'o-', label='Vraies valeurs')
plt.plot(dates_last, y_pred_lstm, 's--', label='LSTM')
plt.plot(dates_last, y_pred_qru, 'd--', label='QLSTM')
plt.title('Comparaison vraie vs prédit (30 derniers jours)')
plt.xlabel('Date')
plt.ylabel('Niveau d\'eau (water_level)')
plt.legend()
plt.tight_layout()
plt.show()


=== 1. Chargement et nettoyage ===

=== LSTM Classique ===


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 32)                  │           4,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,641 (18.13 KB)

 Trainable params: 4,641 (18.13 KB)

 Non-trainable params: 0 (0.00 B)

None

--- Entraînement LSTM classique (avec scheduled sampling) ---

Epoch 1/10, scheduled_sampling_prob=1.000
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 123.8628 - mae: 10.9116 - val_loss: 7.5527 - val_mae: 2.7190

Epoch 2/10, scheduled_sampling_prob=0.950
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.2404 - mae: 2.0672 - val_loss: 0.3194 - val_mae: 0.4614

Epoch 3/10, scheduled_sampling_prob=0.900
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.7256 - mae: 0.6104 - val_loss: 0.1529 - val_mae: 0.3233

Epoch 4/10, scheduled_sampling_prob=0.850
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5170 - mae: 0.5546 - val_loss: 0.1721 - val_mae: 0.3403

Epoch 5/10, scheduled_sampling_prob=0.800
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5838 - mae: 0.5951 - val_loss: 0.1704 - val_mae: 0.3387

Epoch 6/10, scheduled_sampling_prob=0.750
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.5797 - mae: 0.5940 - val_loss: 0.1675 - val_mae: 0.3360

Epoch 7/10, scheduled_sampling_prob=0.7